## Testing successful functions

In [1]:
import openai
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
openai.api_key

'sk-proj-96tqnsajeecaN0t414OZT3BlbkFJ7nJgPclm7TZYxavHCczX'

In [68]:
import json

with open("data/altered_data/all_subjects.json", "r") as file:
    subj_json = json.load(file)
file_length = 0
for subj, keys in subj_json.items():
    file_length += len(keys["admissions"])
print(file_length)

1252


In [47]:
import pandas as pd 
df=pd.read_pickle("data/altered_data/input_data.pkl")



In [204]:
test_df = pd.read_pickle("data/altered_data/predicted_icd_data_temp.pkl")
test_df

,subject_id,admit_id,predicted_icd,predictive_reasoning
0,10001884,24746267,[I130],[The patient has been admitted with a DRG type...


In [203]:
test_df.iloc[0,-1]

["The patient has been admitted with a DRG type of chronic obstructive pulmonary disease (COPD) with severity and mortality scores of 3.0, indicating a significant condition involving the heart and lungs. The medications administered, such as Isosorbide Mononitrate, PredniSONE, and various nebulized treatments, suggest management of heart failure and pulmonary issues. The ICD code I130 is chosen as it encompasses hypertensive heart and chronic kidney disease with heart failure, which aligns with the severity of the patient's condition and the administered medications."]

In [48]:
test_df = pd.read_pickle("data/altered_data/predicted_icd_data_temp.pkl")
final_df = (
    test_df
    .astype({"subject_id":"int", "admit_id":"int"})
    .merge(
        df[["subject_id", "hadm_id", "icd_code"]]
           .groupby(["subject_id", "hadm_id"])
           .icd_code
           .apply(lambda x: x.tolist())
           .reset_index(), 
        left_on=["subject_id","admit_id"], right_on=["subject_id", "hadm_id"], how="left"
           )
    .drop("hadm_id", axis=1)
    .rename(columns={"icd_code":"original_icd_code"})
    )
final_df['is_correct'] = final_df.apply(lambda row: row['predicted_icd'] in row['original_icd_code'], axis=1)
correct_predictions = final_df['is_correct'].sum()

# Display the result
print(f"Total correct predictions: {correct_predictions}/{len(final_df)}")

,subject_id,admit_id,predicted_icd
0,10001884,24746267,I130
1,10001884,24962904,I130
2,10001884,26202981,N390
3,10001884,27507515,N390
4,10002528,28605730,D649
...,...,...,...
75,10049041,23314477,I130
76,10051043,26948064,I130
77,10051043,29090306,M810
78,10051074,28928117,I130


In [66]:
print(f"Total correct predictions: {correct_predictions}/{len(final_df)}")
final_df

Total correct predictions: 31/80


,subject_id,admit_id,predicted_icd,original_icd_code,is_correct
0,10001884,24746267,I130,[D649],False
1,10001884,24962904,I130,[D649],False
2,10001884,26202981,N390,[D649],False
3,10001884,27507515,N390,[D649],False
4,10002528,28605730,D649,[D649],True
...,...,...,...,...,...
75,10049041,23314477,I130,[D649],False
76,10051043,26948064,I130,[D649],False
77,10051043,29090306,M810,[D649],False
78,10051074,28928117,I130,[D649],False


## Testing Data Prep

In [1]:
import pandas as pd
import numpy as np
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI
import os



os.environ["OPENAI_API_KEY"] = "sk-proj-96tqnsajeecaN0t414OZT3BlbkFJ7nJgPclm7TZYxavHCczX"

LLM = ChatOpenAI(
        model="gpt-3.5-turbo-16k",
        temperature=0,
        request_timeout=450,
        max_retries=3,
    )


In [3]:
data = df[
    (df.subject_id == 10532095   )
    # & (df.hadm_id == 22045511)
    ]
# Cleaning final text for patient 10010663 admission number 22209635

In [4]:
data

,icd_code,icd_version,long_title,chapter,block,subject_id,hadm_id,seq_num,gender,anchor_age,...,admission_location,drg_data,medication,labevents_count,abnormal_events_count,merged_labevents_text,microevent_count,merged_microbiology_events,pharmacy_count,merged_pharmacy_events
706,D649,10,"Anemia, unspecified",3 - Diseases of the blood and blood-forming or...,Aplastic and other anemias and other bone marr...,10532095,24112824,20,M,51,...,EMERGENCY ROOM,"[[APR, INFECTIOUS & PARASITIC DISEASES INCLUDI...",[[Calcium Gluconate sliding scale (Critical Ca...,1316.0,674.0,"[Blood-Base Excess-No comment, Blood-Calculate...",15.0,[URINE-Legionella Urinary Antigen -NEGATIVE FO...,153.0,[Vancomycin-IV Piggyback-Inactive (Due to a ch...


In [24]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import re

def chunk_text(text, max_tokens, final=False):
    if final:
        all_other_chunks = text
    else:
        first_chunk, all_other_chunks = text.split('::: ')[0], " ".join(text.split('::: ')[1:])
    
    sentences = re.split(r'(?<=[.!?])\s+', all_other_chunks)

    chunks = []
    current_chunk = ""

    for sentence in sentences:
        # If adding another sentence would exceed the chunk size, store the current chunk
        if len(current_chunk) + len(sentence) + 1 > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            current_chunk += " " + sentence

    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())

    if final:
        return chunks
    else:
        return first_chunk, chunks

def summarising_chunks(patient_information, admission_information, text, max_token_size):
    # Split text into chunks
    first_chunk, chunks = chunk_text(text, max_token_size)

    # Convert chunks to embeddings
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(chunks, embeddings)

    # Query to retrieve relevant chunks
    query = f"Give a summary on each chunk and talk about what the data consists of. Do not just list the input data: {patient_information} and admission information: {admission_information}?"
    relevant_chunks = vectorstore.similarity_search(query)

    # Generate the response based on retrieved chunks
    response = LLM(f"Answer the query based on these chunks, ensuring there is an aggregation for similarities i.e. if there is medicine prescribed then give a summary about all the medicine prescribed, describing them rather than mentioning them. DO NOT LIST OUT DATA POINTS: {relevant_chunks}")
    
    new_summary = first_chunk + response.content
    return new_summary

def final_output_chunks(patient_information, admission_information, text):
    system_role = "You are a helpful clinical coder and your task is to sumamrise the following data into an understandable paragraph that can be used later for interpretation. The data has already been summarised into its own condensed summaries due to the tokenisation limitation."
    human_prompt = "Here is some information on the 'patient_information'= {patient_information}.\nHere is some information on the 'admission_information'={admission_information}.\nHere is what you need to condense into readable paragraphs ensuring that you remove any duplication and retaining all information: 'chunked_text'={chunk}"

    prompt = ChatPromptTemplate(
        messages=[
            SystemMessage(content=system_role),
            HumanMessagePromptTemplate.from_template(
                human_prompt
            ),
        ],
    ) 
    chunked_lists = chunk_text(text, max_tokens=9000, final=True)
    summarised_chunks = ""
    for chunk in chunked_lists:
        _input = prompt.format_prompt(
            patient_information=patient_information, 
            admission_information=admission_information,
            chunk=chunk, 
        )
        response = LLM(_input.to_messages())
        summarised_chunks += response.content + "\n"
    return summarised_chunks

def cleaning_final_output(subject_information, admission_details, text, max_retries=3):
    system_role = "You are a helpful clinical coder and your task is to sumamrise the following data into an understandable paragraph that can be used later for interpretation."
    human_prompt = "Here is some information on the 'patient_information'= {patient_information}.\nHere is some information on the 'admission_information'={admission_information}.\nHere is what you need to condense into readable paragraphs ensuring that you remove any duplication and retaining all information: 'format_text'={format_text}"


    prompt = ChatPromptTemplate(
        messages=[
            SystemMessage(content=system_role),
            HumanMessagePromptTemplate.from_template(
                human_prompt
            ),
        ],
    )


    retries = 0 
    success = False
    # Now call the LLM and get a plain response
    while retries < max_retries and not success:
        try:
            _input = prompt.format_prompt(
                patient_information=subject_information, 
                admission_information=admission_details,
                format_text=text, 
            )
            response = LLM(_input.to_messages())
            success = True
            return response.content
        except Exception as e:
            retries += 1
            print(f"Error encountered - breaking text into chunks and summarising as one.:{e}")
            summarised_outputs = final_output_chunks(
                patient_information=subject_information,
                admission_information=admission_details,
                text=text
                )
            _input = prompt.format_prompt(
                patient_information=subject_information, 
                admission_information=admission_details,
                format_text=summarised_outputs, 
            )
            response = LLM(_input.to_messages())
            return response.content
    if not success:
        raise Exception(f"Failed to generate summary after {max_retries} attempts.")

        



In [6]:
long_text_length=10000
all_subjects = {}
for index, row in data.iterrows():
    # print(f"Subject id: {row.subject_id} - Admission ID: {row.hadm_id}")
    subject_information = f"""Patient is a {"male" if row.gender == "M" else "female"}, {row.anchor_age} years old."""
    admission_details = f"""Patient has been admitted as {row.admission_type} at {row.admission_location} and discharged at {row.discharge_location}."""
    drg_details = """Diagnoses Related Groups::: """
    medication = "The patient is prescribed with the following medication::: "
    labevents = f"The patient has occured in {0 if pd.isna(row.labevents_count) else int(row.labevents_count)} labratory events of which {0 if pd.isna(row.abnormal_events_count) else int(row.abnormal_events_count)} were abnormal. The following were commented on the patients labratory events::: "
    microbiology = f"The patient has had {0 if pd.isna(row.microevent_count) else int(row.microevent_count)} microbiology tests. The following comments on the tests are as follows::: "
    pharmacy = f"The patient has been prescribed the following medicines by the pharmacy: {0 if pd.isna(row.pharmacy_count) else int(row.pharmacy_count)}. In more details::: "

    if type(row.drg_data) == list:
        for drg in row.drg_data:
            drg_details += f"""The patient has a DRG type of {drg[0]} - {drg[1]} - with severity score of {"unknown" if pd.isna(drg[2]) else drg[2]} and mortality score of {"unknown" if pd.isna(drg[3]) else drg[3]}. """

        if len(drg_details) > long_text_length:
            print(f"Creating a condensed summary for DRG of text length:{len(drg_details)} for patient {row.subject_id} admission number {row.hadm_id}")
            new_drg_details = summarising_chunks(
                patient_information=subject_information, 
                admission_information=admission_details, 
                text=drg_details,
                max_token_size=7000
                )
            if len(new_drg_details) > len(drg_details):
                print("Condensed summary is larger than original, keeping original")
                new_drg_details = drg_details
            
        else:
            new_drg_details = drg_details
        
    else:
        new_drg_details = drg_details + "No Details. "
    
    if type(row.medication) == list:
        for med in row.medication:
            medication += f"""{"" if pd.isna(med[1]) else med[1]} - {"" if pd.isna(med[0]) else med[0]}. """
        if len(medication) > long_text_length:
            print(f"Creating a condensed summary for Medication of text length:{len(medication)} for patient {row.subject_id} admission number {row.hadm_id}")
            new_medication = summarising_chunks(
                patient_information=subject_information, 
                admission_information=admission_details, 
                text=medication,
                max_token_size=7000
                )
            if len(new_medication) > len(medication):
                print("Condensed summary is larger than original, keeping original")
                new_medication = medication
        else:
            new_medication = medication
    else:
        new_medication = medication + "No Details. "

    if type(row.merged_labevents_text) == list:
        for labtxt in row.merged_labevents_text:
            labevents += f"Fluid: {labtxt.split('-')[0]} Label: {labtxt.split('-')[1]} Comments: {labtxt.split('-')[2]}. "
        if len(labevents) > long_text_length:
            print(f"Creating a condensed summary for Labevents of text length:{len(labevents)} for patient {row.subject_id} admission number {row.hadm_id}")

            new_labevents = summarising_chunks(
                patient_information=subject_information, 
                admission_information=admission_details, 
                text=labevents,
                max_token_size=7000
                )
            if len(new_labevents) > len(labevents):
                print("Condensed summary is larger than original, keeping original")
                new_labevents = labevents
        else:
            new_labevents = labevents
        
    else:
        new_labevents = labevents + "No Details. "

    if type(row.merged_microbiology_events) == list:
        for mb in row.merged_microbiology_events:
            microbiology += f"Organism name: {mb.split('-')[0]}, Antibiotic: {mb.split('-')[1]}, Comments: {mb.split('-')[2]}. "
        if len(microbiology) > long_text_length:
            print(f"Creating a condensed summary for Microbiology of text length:{len(microbiology)} for patient {row.subject_id} admission number {row.hadm_id}")

            new_microbiology = summarising_chunks(
                patient_information=subject_information, 
                admission_information=admission_details, 
                text=microbiology,
                max_token_size=7000
                )
            if len(new_microbiology) > len(microbiology):
                print("Condensed summary is larger than original, keeping original")
                new_microbiology = microbiology
        else:
            new_microbiology = microbiology
        
    else:
        new_microbiology = microbiology + "No Details. "


    if type(row.merged_pharmacy_events) == list:
        for pharm in row.merged_pharmacy_events:
            pharmacy += f"Medication: {pharm.split('-')[0]}, Proc type: {pharm.split('-')[1]}, Status: {pharm.split('-')[2]}, Frequency: {pharm.split('-')[3]}, Dispensation: {pharm.split('-')[4]}. "
        if len(pharmacy) > long_text_length:
            print(f"Creating a condensed summary for Pharmacy of text length:{len(pharmacy)} for patient {row.subject_id} admission number {row.hadm_id}")

            new_pharmacy = summarising_chunks(
                patient_information=subject_information, 
                admission_information=admission_details, 
                text=pharmacy,
                max_token_size=7000
                )
            if len(new_pharmacy) > len(pharmacy):
                print("Condensed summary is larger than original, keeping original")
                new_pharmacy = pharmacy
        else:
            new_pharmacy = pharmacy
        
    else:
        new_pharmacy = pharmacy + "No Details. "


    text = admission_details + new_drg_details + new_medication + new_labevents + new_microbiology + new_pharmacy
    if len(text) > 20000:
        print(f"Cleaning final text for patient {row.subject_id} admission number {row.hadm_id} because the text has length {len(text)}")
        final_text = cleaning_final_output(subject_information, admission_details, text)
    else:
        final_text = text
    if row.subject_id not in all_subjects.keys():            
        all_subjects[row.subject_id] = {
            "subject_details": subject_information,
            "admissions": [{row.hadm_id: final_text}]
            
        }
    else:
        all_subjects[row.subject_id]["admissions"].append({row.hadm_id: final_text})

Creating a condensed summary for Medication of text length:25093 for patient 10532095 admission number 24112824


C:\Users\waila\AppData\Local\Temp\ipykernel_21268\3979949700.py:32: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()
C:\Users\waila\AppData\Local\Temp\ipykernel_21268\3979949700.py:50: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = LLM(base_prompt)


Creating a condensed summary for Labevents of text length:70051 for patient 10532095 admission number 24112824
Creating a condensed summary for Pharmacy of text length:19405 for patient 10532095 admission number 24112824
Cleaning final text for patient 10532095 admission number 24112824 because the text has length 65577


NameError: name 'BadRequestError' is not defined

In [9]:
print(len(admission_details))
print(len(new_drg_details))
print(len(new_medication))
print(len(new_labevents))
print(len(new_microbiology))
print(len(new_pharmacy))
print(len(text))


82
329
24721
37868
1703
887
65590


In [12]:
print(new_medication)

The patient is prescribed with the following medicationThe following medications were prescribed:

- Calcium Gluconate sliding scale (Critical Care-Ionized calcium)
- Octreotide Acetate
- Nicotine Patch
- Lactulose
- MetroNIDAZOLE
- Albuterol Inhaler
- Ipratropium Bromide MDI
- NORepinephrine
- Fentanyl Citrate
- Propofol
- Sodium Chloride 0.9% Flush
- Albumin 5% (25g / 500mL)
- Potassium Chloride Replacement (Critical Care and Oncology)
- Magnesium Sulfate Replacement (Critical Care and Oncology)
- Chlorhexidine Gluconate 0.12% Oral Rinse
- Pantoprazole
- Azithromycin
- Rifaximin
- CefTAZidime
- Midodrine
- CefTAZidime
- Potassium Chloride
- Furosemide
- Piperacillin-Tazobactam
- Dexmedetomidine
- Heparin
- Lidocaine Jelly 2% (Urojet)
- QUEtiapine Fumarate
- Sodium Phosphate
- Phytonadione
- QUEtiapine Fumarate
- Escitalopram Oxalate
- TraZODone
- GuaiFENesin
- Calcium Gluconate sliding scale (Critical Care-Ionized calcium)
- Haloperidol
- Lidocaine 1% (For PICC/Midline Insertions)
- 

## Testing LLM Coder

In [160]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema import SystemMessage
from langchain_openai import ChatOpenAI
from typing import List, Dict
from datetime import datetime
import logging
import pandas as pd
import json
import os 


class Admission(BaseModel):
    admission_id: str = Field(description="The ADMISSION ID of the patients visit")
    icd_code: str = Field(description="The diagnosis code assigned to the patients visit using the ICD definitions provided. a python string consisting of the diagnosis code (Only the ICD code e.g. B12.8  & not the definition)")
    predictive_reasoning: str = Field(description="The reason as to why the ICD code prediction was made.")

class Patient(BaseModel):
    __root__: Dict[str, List[Admission]]

def get_icd_llm(icd_definitions, subject_id, subject_details, admission_id, admission_details):
    system_role = "You are a helpful clinical coder and your task is to use the following ICD definitions and assign diagnosis codes (ICD codes) from the notes and data provided. The user will provide text and and your role is to assign a code."
    human_prompt = "Predict an ICD code for the following subject strictly based on the admission information. The following ICD codes and definitions is all that you are allowed to use {icd_definitions}\n\nDO NOT USE CODES THAT ARE NOT LISTED ABOVE. ENSURE THAT THE ICD_CODE OUTPUT IS LESS THAN 6 CHARACTERS AND ONLY CONTAINS THE CODE NOT THE DEFINITION.\nHere is some information on the Patient: subject_id={subject_id} - {subject_details}.\nHere is information on the patients admission details (admission_id={admission_id}) that requires a diagnosis code: {admission_details}"
    # group_schema = "output must be a python dictionary of a single key 'subject_id' and the value must be a python dictionary. In this python dictionary, each key must be the 'admission_id' and the value must be a python string consisting of the diagnosis code (Only the ICD code e.g. B12.8  & not the definition)"



    structured_llm = LLM.with_structured_output(Patient, method="json_mode")
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessage(content=system_role),
            HumanMessagePromptTemplate.from_template(
                human_prompt
            ),
        ],
    )

    _input = prompt.format_prompt(
        icd_definitions=icd_definitions, 
        subject_id=subject_id,
        subject_details=subject_details, 
        admission_id=admission_id,
        admission_details=admission_details,
    )

    icd_def_codes = [word.split(": ")[1].split(" - ")[0] for word in icd_definitions.split('\n') if word.startswith("ICD code")]
    success=False
    retries = 0
    while not success and retries < 5:
        try:
            base_prompt = f"Using the following commands, respond in JSON with the 'subject_id' as keys and the values are a list of JSONs with 'admission_id' and 'icd_code' and 'predictive_reasoning' as keys. {_input.to_messages()}"
            output = structured_llm.invoke(base_prompt)
            if type(output.__root__) is dict:
                codes = []
                for adm in output.__root__[subj_id]:
                    codes.append(adm.icd_code)
                if any(x in icd_def_codes for x in codes):
                    success = True
                    logging.info("Success")
                    return output
                else:
                    logging.info("Didn't output a correct ICD code")
                    retries += 1
                    base_prompt = "IMPORTANT: ENSURE THAT YOU OUTPUT AN ICD CODE FROM THE DEFINITIONS BELOW. " + base_prompt
            else:
                logging.info("Output was not of dictionary type")
                retries += 1
                base_prompt = base_prompt + "IMPORTANT: ENSURE THAT THE OUTPUT IS A DICTIONARY"
            
        except Exception as e:
            logging.info(f"Retrying as output was not dict - Error: {e}")
            retries += 1
    if not success:
        logging.info("Tried more than 5 times ")


def get_output(all_subjects, icd_definitions, logging):
    df_output = pd.DataFrame(columns=["subject_id", "admit_id", "predicted_icd"])
    for subj_id, subj_info in all_subjects.items():
        # Do whatever here with subject id
        for admission in subj_info["admissions"]:
            for admit_id, admit_info in admission.items():
                logging.info(f"Predicting ICD for Patient {subj_id} with Admission ID: {admit_id}")
                each_output = get_icd_llm(
                        icd_definitions=icd_definitions, 
                        subject_id=subj_id,
                        subject_details=subj_info["subject_details"], 
                        admission_id=admit_id,
                        admission_details=admit_info,
                )
            codes = []
            for adm in each_output.__root__[subj_id]:
                codes.append(adm.icd_code)

            df_output = df_output._append({
                "subject_id":subj_id,
                "admit_id":admit_id,
                "predicted_icd":codes
                }, ignore_index=True)
                # df_output.to_pickle("data/altered_data/predicted_icd_data_temp.pkl")
    # df_output.to_pickle("data/altered_data/predicted_icd_data.pkl")
    return df_output



In [71]:
with open("data/altered_data/icd_defintions.txt", "r") as file:
    icd_definitions = file.read()

with open('data/altered_data/all_subjects.json', 'r') as fp:
    all_subjects = json.load(fp)

In [113]:
test_subject = {k:all_subjects[k] for k in all_subjects.keys() if k == "10055694"}
test_output = get_output(test_subject, icd_definitions, logging)

['D649', 'E7800', 'I130', 'M810', 'N390']

In [118]:
df_output = pd.DataFrame(columns=["subject_id", "admit_id", "predicted_icd"])
for subj_id, subj_info in test_subject.items():
    # Do whatever here with subject id
    for admission in subj_info["admissions"]:
        for admit_id, admit_info in admission.items():
            print(f"Predicting ICD for Patient {subj_id} with Admission ID: {admit_id}")
            each_output = get_icd_llm(
                    icd_definitions=icd_definitions, 
                    subject_id=subj_id,
                    subject_details=subj_info["subject_details"], 
                    admission_id=admit_id,
                    admission_details=admit_info,
            )
            codes = []
            for adm in each_output.__root__[subj_id]:
                codes.append(adm.icd_code)

            df_output = df_output._append({
                "subject_id":subj_id,
                "admit_id":admit_id,
                "predicted_icd":codes
                }, ignore_index=True)

Predicting ICD for Patient 10055694 with Admission ID: 21758998
Predicting ICD for Patient 10055694 with Admission ID: 26630138
Predicting ICD for Patient 10055694 with Admission ID: 27416032


In [120]:
each_output

Patient(__root__={'10055694': [Admission(admission_id='27416032', icd_code='unknown')]})

In [99]:
each_output.__root__["10055694"]

[Admission(admission_id='21758998', icd_code='D649'),
 Admission(admission_id='21758998', icd_code='E7800'),
 Admission(admission_id='21758998', icd_code='I130'),
 Admission(admission_id='21758998', icd_code='M810'),
 Admission(admission_id='21758998', icd_code='N390')]

In [100]:
# [code for i in range(len(each_output.__root__["10055694"])) for code in each_output.__root__["10055694"][i].icd_code ]


['D649', 'E7800', 'I130', 'M810', 'N390']

In [88]:
type(each_output.__root__) is dict

True

'Patient has been admitted as OBSERVATION ADMIT at EMERGENCY ROOM and discharged at HOME.Diagnoses Related Groups::: The patient has a DRG type of APR - OTHER DIGESTIVE SYSTEM DIAGNOSES - with severity score of 3.0 and mortality score of 3.0. The patient has a DRG type of HCFA - G.I. HEMORRHAGE W MCC - with severity score of unknown and mortality score of unknown. The patient is prescribed with the following medication::: Administered - Pantoprazole. Not Given - Fluticasone-Salmeterol Diskus (250/50) . Administered - Nephrocaps. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Administered - Aspirin. Not Given - Calcium Acetate. Administered - Nephrocaps. Administered - Pantoprazole. Administered - Fluticasone-Salmeterol Diskus (250/50) . Administered - Albuterol 0.083% Neb Soln. Administered - Calcium Acetate. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9% 

In [196]:
test_subject

{'10055694': {'subject_details': 'Patient is a female, 58 years old.',
  'admissions': [{'21758998': 'Patient has been admitted as OBSERVATION ADMIT at EMERGENCY ROOM and discharged at HOME.Diagnoses Related Groups::: The patient has a DRG type of APR - OTHER DIGESTIVE SYSTEM DIAGNOSES - with severity score of 3.0 and mortality score of 3.0. The patient has a DRG type of HCFA - G.I. HEMORRHAGE W MCC - with severity score of unknown and mortality score of unknown. The patient is prescribed with the following medication::: Administered - Pantoprazole. Not Given - Fluticasone-Salmeterol Diskus (250/50) . Administered - Nephrocaps. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Flushed - Sodium Chloride 0.9%  Flush. Administered - Aspirin. Not Given - Calcium Acetate. Administered - Nephrocaps. Administered - Pantoprazole. Administered - Fluticasone-Salmeterol Diskus (250/50) . Administered - Albuterol 0.083% Neb Soln. A

Retrying as output was not dict - Error: Error code: 400 - {'error': {'message': "Invalid parameter: 'response_format' of type 'json_object' is not supported with this model.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}
Retrying as output was not dict - Error: Error code: 400 - {'error': {'message': "Invalid parameter: 'response_format' of type 'json_object' is not supported with this model.", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}
Success


In [189]:
df[(df.subject_id == 10055694) & (df.hadm_id == 26630138)]

,icd_code,icd_version,long_title,chapter,block,subject_id,hadm_id,seq_num,gender,anchor_age,...,admission_location,drg_data,medication,labevents_count,abnormal_events_count,merged_labevents_text,microevent_count,merged_microbiology_events,pharmacy_count,merged_pharmacy_events
86,D649,10,"Anemia, unspecified",3 - Diseases of the blood and blood-forming or...,Aplastic and other anemias and other bone marr...,10055694,26630138,1,F,58,...,EMERGENCY ROOM,NaN,"[[Albuterol 0.083% Neb Soln, Administered], [I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
output.__root__

{'10055694': [Admission(admission_id='26630138', icd_code='D649', predictive_reasoning='Anemia, unspecified')]}

In [169]:
codes

['D649', 'E7800', 'I130', 'M810', 'N390']